# What you already know: HOFs and Scala collections

I assume you already know the basics of Scala and higher-order functions, at least as they are used in the Scala Collections library. 

In [1]:
// example of List processing with map, filter, etc. 

List(1,2,3,4)
    .map(_ + 1)
    .filter(_ % 2 == 0)
    .reduce(_ + _)

res0: Int = 6

# Spark: standalone setup

Spark is a distributed processing framework for transforming big data sets using the computational power of a dedicated cluster. But we can use Spark in an standalone mode (i.e. with no cluster at all), for testing or pedagogical purposes. In that case, we just exploit the cores of your local processor.

### Create the Spark session

The Spark session is the entry point to the Spark interpreter. We need it for running Spark programs.

In [2]:
// Create a Spark session in standalone mode

import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

val spark: SparkSession = 
    NotebookSparkSession
      .builder()
      .master("local[*]")
      .getOrCreate()


Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/02/24 14:48:34 INFO SparkContext: Running Spark version 2.4.5
20/02/24 14:48:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
20/02/24 14:48:35 INFO SparkContext: Submitted application: 4eaeccd3-cb24-40b8-a2f9-5733373d9f75
20/02/24 14:48:35 INFO SecurityManager: Changing view acls to: jovyan
20/02/24 14:48:35 INFO SecurityManager: Changing modify acls to: jovyan
20/02/24 14:48:35 INFO SecurityManager: Changing view acls groups to: 
20/02/24 14:48:35 INFO SecurityManager: Changing modify acls groups to: 
20/02/24 14:48:35 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
20/02/24 14:48:35 INFO Utils: Successfully started service 'sparkD

20/02/24 14:48:37 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://0e8a1150599b:36145/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1582555717163
20/02/24 14:48:37 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://0e8a1150599b:36145/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1582555717164
20/02/24 14:48:37 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.2.0/scala-xml_2.12-1.2.0-sources.jar at spark://0e8a1150599b:36145/jars/scala-xml_2.12-1.2.0-sources.jar with timestamp 1582555717164
20/02/24 14:48:37 INFO SparkContext: Added JAR file:/home/jovyan/.cache

20/02/24 14:48:37 INFO BlockManagerMaster: Registered BlockManager BlockManagerId(driver, 0e8a1150599b, 37475, None)
20/02/24 14:48:37 INFO BlockManager: Initialized BlockManager: BlockManagerId(driver, 0e8a1150599b, 37475, None)


Spark UI

import $ivy.$                                   

import $ivy.$                              


import org.apache.spark.sql.{NotebookSparkSession, SparkSession}


spark: SparkSession = org.apache.spark.sql.SparkSession@19a968a6

### Logging configuration

This is convenient to minimize the amount of info displayed in the terminal.

In [3]:
import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}
Logger.getRootLogger().setLevel(Level.ERROR)

import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


### Useful imports

In [4]:
import spark.implicits._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._

import spark.implicits._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._

# Your first Spark program 

The following program is an example of a _Dataset_ program. The Dataset API is one of the languages for distributed data processing that the Spark framework provides. We will omit reference in this notebook to other APIs such as RDDs, DataFrames, etc.

In [5]:
Logger.getRootLogger().setLevel(Level.ERROR)

In [6]:
List(1,2,3,4).toDS
    .map(_ + 1)
    .filter(i => i % 2 == 0)
    .reduce(_ + _)

reduce at cmd5.sc:4

1 / 1

res5: Int = 6

In principle, this Dataset program does not differ significantly from the Scala collection program. Syntactically, the only difference appears to be the `.toDS` expression: 

In [ ]:
List(1,2,3,4).toDS

But, of course, there are several major differences.

# First difference: performance

Let's define a heavy computation:

In [7]:
def heavyComp(ms: Int = 1000)(x: Int): Int = {
  Thread.sleep(ms)
  x+1
}

defined function heavyComp

and a way to measure execution time:

In [8]:
def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

defined function run

In [ ]:
run(println("hola"))

The following Scala Collection program takes some time to execute:

In [ ]:
run{
    List(1,2,3,4).map(heavyComp(): Int => Int).reduce(_ + _)
}

However, the equivalent Dataset program takes half time (or less time depending on the number of cores of your processor)!

In [ ]:
run(
    List(1,2,3,4).toDS.map(heavyComp()).reduce(_ + _)
)

The Dataset program run faster because the Spark framework is designed to take advantage of the parallel and distributed architecture of your computing infrastructure. In our case, it simply exploits the number of cores of your processor.

However, note that using Spark to parallelize your code is overkill. If you are not in a truly distributed setting, you can get along the same benefits more simply using the parallel collections framework of the Scala standard library: 

In [ ]:
run(List(1,2,3,4).par.map(heavyComp()).reduce(_ + _))

# Second difference: _laziness_

Let's compare this Scala collection transformation:

In [ ]:
val result: List[Int] = List(1,2,3,4).map(_ + 1)

with the following Dataset one:

In [ ]:
List(1,2,3,4).toDS.map(_ + 1).filter(_ % 2 == 0).reduce(_ + _)

In [ ]:
val program: Dataset[Int] = List(1,2,3,4).toDS.map(_ + 1)

We obtain no transformation at all! Dataset programs are that: _programs_. We won't find any data in an instance of `Dataset`, just a program or _generator_ of a data set. A `Dataset` declares a number of _transformations_ that will be eventually enacted with specific _actions_. For instance, using `collect`:

In [ ]:
val result: Array[Int] = program.collect
val result2: Int = program.reduce(_ + _)

or `reduce`:

In [ ]:
val result2: Int = program.reduce(_ + _)

Spark Datasets are said to be *lazy*, because we don't inmediately obtain an answer. Rather, we _declaratively_ specify a number of transformations to be applied, and wait until a specific action interprets the transformation program to obtain the desired result. And the same program may be interpreted differently: we may simply want to execute the transformations using `collect`, or may want to perform some calculation using `reduct`. This difference between _transformations_ and _actions_ is reflected very precisely in the [Dataset API](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset).  

### Inspecting the structure of a Spark program

A `Dataset` is a program that it's compiled into a lower-level program before it can be actually executed. The compiler of datasets is called _catalyst_. We can inspect the execution plan that is generated for a particular dataset using `explain`:

In [ ]:
List(1,2,3,4).toDS.map(_ + 1).explain

The execution plan of a dataset is in turn compiled into an `RDD`, i.e. a lower-level abstraction program:

In [ ]:
// RDD = Resilient Distributed Dataset
val rdd: org.apache.spark.rdd.RDD[Int] = List(1,2,3,4).toDS.map(_ + 1).rdd
rdd.toDebugString

### Spark and functional programming

This distinction lies also at the heart of functional programming. On the one hand, there are _programs_ written in a DSL. On the other, there are _interpreters_ that run this program according to different semantics. This is also reflected in the Scala Collections API, particularly, in the notion of _views_. For instance, the following transformation is similarly _lazy_: 

In [ ]:
// List(1,2,3,4).map(_ + 1).filter(_ % 2 == 0)

In [ ]:
List(1,2,3,4).view.map(heavyComp())
    .filter{ i => 
        println("hola");
        i % 2 == 0
    }

The transformation is only executed when we execute the view using, e.g. `toList`: 

In [ ]:
List(1,2,3,4).view.map(heavyComp()).toList

`SeqView`s are *programs*, much in the same way as `Dataset` objects, whereas `toList` is an *action*, equivalent to the `Dataset` actions `collect` and `reduce`.

Similarly, Scala _iterators_ are also good examples of *lazyness*. When we create an iterator from a collection, as in:

In [ ]:
val it: Iterator[Int] = List(1,2,3,4).iterator

and specify a number of transformations:

In [ ]:
val it2: Iterator[Int] = List(1,2,3,4).iterator
    .map{i => println("it"); i + 1}
    .filter{ i => println("it2"); i % 2 == 0 }
    //.toList

we don't inmediately obtain those transformations. We are just creating a new iterator that will generate the correspoding data when we ask for it:

In [ ]:
it2.toList

Actually, there is a close relationship between Spark RDDs (the transformation language in which `Dataset`s are actually translated into), and iterators. 

# Third difference: the execution framework

When an action is applied on a `Dataset` program a _job_ is executed by the distributed platform of Spark through a sequence of *stages*; in each stage, the work to be done is performed concurrently by a number of _tasks_. 

The so-called [Spark UI](http://localhost:4040/) allows us to debug the execution process of all the jobs that are submitted for execution through a given Spark session. For instace, the following action launches a job that can be inspected through the Spark UI: 

In [9]:
val ds: Dataset[Int] = List(1,2,3,4).toDS.map(heavyComp())

ds: Dataset[Int] = [value: int]

In [ ]:
ds.explain

In [ ]:
ds.collect

Each bar in the notebook execution corresponds to one stage of the job exectuion; the X/Y label in each bar indicates the number of tasks already executed (X) and the total number of tasks of that stage (Y). 

We can get the work performed by tasks in each partition through `foreachPartition`:

In [10]:
ds.foreachPartition{ it : Iterator[Int] => 
    println(s"Task output: " + it.toList)
}

foreachPartition at cmd9.sc:1

1 / 1

Task output: List(2, 3, 4, 5)


In [11]:
def collectPartition[T](ds: Dataset[T]) : Unit =
    ds.foreachPartition{ it: Iterator[T] =>
        println("Task output: " + it.toList)
    }

defined function collectPartition

In [ ]:
collectPartition(ds)

We will use this action quite frequently, so let's define an *extension method* for the `Dataset` type:

In [12]:
// Exxtension de metodos, solo un argumento: el objeto que quieres transformar
implicit class DatasetOps[T](ds: Dataset[T]){
    def collectPartitions: Unit = 
        ds.foreachPartition{it : Iterator[T] => println(it.toList)}
}

defined class DatasetOps

The number of tasks scheduled for each stage depends on the number of partitions associated to the dataset. When the dataset is first created from a Scala collection, the number of partitions defaults to the number of cores specified when the Spark context was created. 

In [ ]:
List(1,2,3,4).toDS.rdd.getNumPartitions

The number of partitions can be set to a specific value using `repartition`: 

In [13]:
List(1,2,3,4,5,6,7,8,9,10,11,12).toDS
    .repartition(24)
    .map(heavyComp(2000))
    .collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

24 / 24

List()
List()
List()
List()
List()
List()
List()
List()
List()
List()
List()
List()
List(12)
List(4)
List(8)
List(11)
List(6)
List(7)
List(9)
List(3)
List(10)
List(5)
List(2)
List(13)


# Shuffling: narrow vs. wide transformations
//narrow: particiones de datos independientes
//wide transformations: particiones con datos dependiente a datos de otras particiones

Note that a new stage is created when the dataset is repartitioned. More commonly, new stages are created when so-called _wide_ transformations are interpreted. _Narrow_ transformations are those transformations which are not wide: `map`, `filter`, etc. For instance, this program will execute in one stage: 

In [14]:
List(("a", 1), ("b", 2), ("a", 3), ("d", 3), ("b", 4)).toDS
    .map{ case (key, value) => (key, value + 1) }
    .collect

collect at cmd13.sc:2

1 / 1

res13: Array[(String, Int)] = Array(
  ("a", 2),
  ("b", 3),
  ("a", 4),
  ("d", 4),
  ("b", 5)
)

and the following one as well:

In [15]:
List(("a", 1), ("b", 2), ("a", 3), ("d", 3), ("b", 4)).toDS
    .map{ case (key, value) => (key, value + 1) }
    .filter{ t => t._1 == "a" }
    .collect

collect at cmd14.sc:3

1 / 1

res14: Array[(String, Int)] = Array(("a", 2), ("a", 4))

However, this one introduces a new stage:

In [16]:
List(("a", 1), ("b", 2), ("a", 3), ("d", 3), ("b", 4)).toDS
    .map{ case (key, value) => (key, value + 1) }
    .groupByKey(_._1)
    .mapGroups((key, values) => (key, values.toList.map(_._2)))
    .collect

collect at cmd15.sc:4

1 / 1

collect at cmd15.sc:4

200 / 200

res15: Array[(String, List[Int])] = Array(
  ("d", List(4)),
  ("b", List(3, 5)),
  ("a", List(2, 4))
)

Why? Which difference between `filter` and `groupBy` creates such a need for a new stage? And why the next stage generates a dataset with 200 partitions? Let's answer these questions: 
* First, a new stage is created when data needs to be moved, or *shuffled*, between partitions. 
* Indeed, this is the case for `groupBy`.
* Last, 200 is the default number of partitions created when a shuffled is needed.

This value can be customised as follows:

In [19]:
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [28]:
List(("a", 1), ("b", 2), ("a", 3), ("d", 3), ("b", 4)).toDS
    .map{ case (key, value) => (key, value + 1) }
    .groupByKey(_._1)
    .mapGroups((key, values) => (key, values.toList.map(_._2)))
    .collect

collect at cmd27.sc:4

1 / 1

collect at cmd27.sc:4

8 / 8

res27: Array[(String, List[Int])] = Array(
  ("b", List(3, 5)),
  ("a", List(2, 4)),
  ("d", List(4))
)

We can inspect the contents of the different partitions after each transformation:

In [17]:
List(("a", 1), ("e", 2), ("f", 3), ("d", 3), 
     ("z", 3), ("k", 3), ("i", 3), ("o", 3), 
     ("l",2), ("b", 2), ("a", 3), ("d", 3), 
     ("b", 4), ("e", 4)).toDS
    .collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List((a,1), (e,2), (f,3), (d,3), (z,3), (k,3), (i,3), (o,3), (l,2), (b,2), (a,3), (d,3), (b,4), (e,4))


In [ ]:
List(("a", 1), ("e", 2), ("f", 3), ("d", 3), ("z", 3), ("k", 3), ("i", 3), ("o", 3), ("l",2), ("b", 2), ("a", 3), ("d", 3), ("b", 4), ("e", 4)).toDS
    .map{ case (key, value) => (key, value + 1) }
    .collectPartitions

In [21]:
List(("a", 1), ("e", 2), ("f", 3), ("d", 3), ("z", 3), ("k", 3), ("i", 3), ("o", 3), ("l",2), ("b", 2), ("a", 3), ("d", 3), ("b", 4), ("e", 4)).toDS
    .map{ case (key, value) => (key, value + 1) }
    .groupByKey(_._1)
    .mapGroups((key, values: Iterator[(String,Int)]) => (key, values.toList.map(_._2)))
    .collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

8 / 8

List()
List()
List()
List()
List((f,List(4)), (k,List(4)), (o,List(4)))
List((b,List(3, 5)), (i,List(4)))
List((a,List(2, 4)))
List((d,List(4, 4)), (e,List(3, 5)), (l,List(3)), (z,List(4)))


How do Spark decides where to move the data?

In [22]:
List(("a", 1), ("e", 2), ("f", 3), ("d", 3), ("z", 3), ("k", 3), ("i", 3), ("o", 3), ("l",2), ("b", 2), ("a", 3), ("d", 3), ("b", 4), ("e", 4)).toDS
    .map{ case (key, value) => (key, value + 1) }
    .groupByKey(_._1)
    .mapGroups((key, values) => (key, values.toList.map(_._2)))
    .explain

== Physical Plan ==
*(2) SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, assertnotnull(input[0, scala.Tuple2, true])._1, true, false) AS _1#159, newInstance(class org.apache.spark.sql.catalyst.util.GenericArrayData) AS _2#160]
+- MapGroups org.apache.spark.sql.KeyValueGroupedDataset$$Lambda$4796/1154500740@232fa730, value#155.toString, newInstance(class scala.Tuple2), [value#155], [_1#151, _2#152], obj#158: scala.Tuple2
   +- *(1) Sort [value#155 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(value#155, 8)
         +- AppendColumnsWithObject ammonite.$sess.cmd21$Helper$$Lambda$4936/1053265561@53818c0b, [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, assertnotnull(input[0, scala.Tuple2, true])._1, true, false) AS _1#151, assertnotnull(input[0, scala.Tuple2, true])._2 AS _2#152], [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.

# Narrow or wide?

The transformations in the Dataset API can be classified into narrow and wide transformations, systematically. We have already mentioned that `map` and `filter` belong to the former, and `groupByKey` to the latter. What about the following ones?

### `coalesce`
//Reducir numero de particiones

In [24]:
val ds = List(1,2,2,3,4,4,5,6,4,7,3,8).toDS

ds: Dataset[Int] = [value: int]

In [29]:
ds.collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List(1, 2, 2, 3, 4, 4, 5, 6, 4, 7, 3, 8)


In [26]:
ds.coalesce(2).collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List(1, 2, 2, 3, 4, 4, 5, 6, 4, 7, 3, 8)


### `dropDuplicates` (eliminar elementos duplicados)
//wide

In [27]:
ds.dropDuplicates.collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

8 / 8

List()
List()
List()
List(6)
List(5)
List(1, 3, 7)
List(2, 4)
List(8)


### `flatMap`
//Narrow

In [30]:
ds.flatMap(i => List(i, -i)).collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List(1, -1, 2, -2, 2, -2, 3, -3, 4, -4, 4, -4, 5, -5, 6, -6, 4, -4, 7, -7, 3, -3, 8, -8)


### `limit` (limitar el nº de elementos de salida)
//Narrow

In [31]:
ds.limit(6).collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List(1, 2, 2, 3, 4, 4)


### `mapPartitions`

In [32]:
ds.mapPartitions{ it: Iterator[Int] => it.map(_ + 1) }.collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List(2, 3, 3, 4, 5, 5, 6, 7, 5, 8, 4, 9)


In [ ]:
implicit class DMap[T](da:Dataset[T])

### `repartition`

In [33]:
ds.repartition(2).collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

2 / 2

List(3, 7, 6, 2, 4, 4)
List(3, 5, 8, 2, 4, 1)


Compare it with `coalesce`:

In [34]:
ds.coalesce(2).collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List(1, 2, 2, 3, 4, 4, 5, 6, 4, 7, 3, 8)


Differences can be "explained":

//Todo elemento de una particion se ha ido a otra particion (una) -> coalesce, por eso narrow

//Elementos de una particion se van a distintos particiones -> repartition, por eso Wide

In [ ]:
ds.repartition(2).explain

In [ ]:
ds.coalesce(2).explain

### `sort`

In [35]:
ds.sort("value").collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

8 / 8

List()
List(1, 2, 2)
List(3, 3)
List(4, 4, 4)
List(5)
List(6)
List(7)
List(8)


In [36]:
ds.sort("value").collect

collect at cmd35.sc:1

1 / 1

collect at cmd35.sc:1

1 / 1

collect at cmd35.sc:1

8 / 8

res35: Array[Int] = Array(1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 7, 8)

# What about transformations that relate several datasets?

Commonly, information is spread across several datasets, and the Spark Dataset API includes transformations to deal with this situation.

### `Union`

In [37]:
val ds1: Dataset[Int] = List(1,2,3,4).toDS.repartition(3)
val ds2: Dataset[Int] = List(5,6,7,8).toDS.repartition(6)

ds1: Dataset[Int] = [value: int]
ds2: Dataset[Int] = [value: int]

In [38]:
ds1.collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

3 / 3

List(4)
List(3, 1)
List(2)


In [39]:
ds2.collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

6 / 6

List()
List()
List(7)
List(5)
List(6)
List(8)


In [40]:
ds1.union(ds2).collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

9 / 9

List()
List()
List(4)
List(3, 1)
List(2)
List(7)
List(5)
List(6)
List(8)


No shuffle involved, just a single stage which makes the union of the different partitions.

### `Join`

In [41]:
object DS{
    case class Person(name: String, age: Int)
    case class Student(name: String, degree: String, year: Int)
}

import DS._

defined object DS
import DS._

In [42]:
val people: Dataset[Person] = List(
    Person("Yihui", 20),
    Person("Noelia", 19),
    Person("Gabriel", 22),
    Person("Javier", 21)).toDS

people: Dataset[Person] = [name: string, age: int]

In [43]:
val students: Dataset[Student] = List(
    Student("Yihui", "II", 2000),
    Student("Yihui", "M", 2001),
    Student("Noelia", "II", 2000),
    Student("Noelia", "IS", 2000),
    Student("Gabriel", "II", 2004),
    Student("Javier", "II", 2005),
    Student("Javier", "M", 2005)).toDS

students: Dataset[Student] = [name: string, degree: string ... 1 more field]

In [44]:
people.collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List(Person(Yihui,20), Person(Noelia,19), Person(Gabriel,22), Person(Javier,21))


In [45]:
students.collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

List(Student(Yihui,II,2000), Student(Yihui,M,2001), Student(Noelia,II,2000), Student(Noelia,IS,2000), Student(Gabriel,II,2004), Student(Javier,II,2005), Student(Javier,M,2005))


In [46]:
people.join(students, "name").collectPartitions

run at ThreadPoolExecutor.java:1149

1 / 1

foreachPartition at cmd11.sc:3

1 / 1

List([Yihui,20,II,2000], [Yihui,20,M,2001], [Noelia,19,II,2000], [Noelia,19,IS,2000], [Gabriel,22,II,2004], [Javier,21,II,2005], [Javier,21,M,2005])


Somewhat unexpectedly, there is no shuffle! This is because Spark performs the join following a "broadcast" strategy:

In [47]:
people.join(students, "name").explain

== Physical Plan ==
*(1) Project [name#226, age#227, degree#233, year#234]
+- *(1) BroadcastHashJoin [name#226], [name#232], Inner, BuildLeft
   :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true]))
   :  +- LocalTableScan [name#226, age#227]
   +- LocalTableScan [name#232, degree#233, year#234]


This happens when one of the datasets is small enough to be copied for each partition. We can force Spark to avoid broadcast as follows (just for testing purposes): 

In [48]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",0)


In [49]:
people.join(students, "name").collectPartitions

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

1 / 1

foreachPartition at cmd11.sc:3

8 / 8

List()
List()
List()
List()
List([Javier,21,II,2005], [Javier,21,M,2005])
List([Gabriel,22,II,2004])
List([Noelia,19,II,2000], [Noelia,19,IS,2000])
List([Yihui,20,II,2000], [Yihui,20,M,2001])


# Caching

One of the most distinctive features of Spark is its ability to cache computations of datasets.

In [50]:
val ds5: Dataset[Int] = (0 to 1000).toDS.map(heavyComp(20))

ds5: Dataset[Int] = [value: int]

In [51]:
run(ds5.count)
run(ds5.count)

count at cmd50.sc:1

1 / 1

count at cmd50.sc:1

1 / 1

Took 21054


...

1 / 1

count at cmd50.sc:2

1 / 1

Took 20846


res50_0: Long = 1001L
res50_1: Long = 1001L

Now with caching:

In [52]:
ds5.cache

res51: Dataset[Int] = [value: int]

In [53]:
run(ds5.count)
run(ds5.count)

count at cmd52.sc:1

1 / 1

count at cmd52.sc:1

1 / 1

Took 21473


...

1 / 1

count at cmd52.sc:2

1 / 1

Took 212


res52_0: Long = 1001L
res52_1: Long = 1001L

We may instruct the Spark interpreter to not use cached data:

In [54]:
ds5.unpersist

res53: Dataset[Int] = [value: int]

In [55]:
run(ds5.count)

count at cmd54.sc:1

1 / 1

count at cmd54.sc:1

1 / 1

Took 20794


res54: Long = 1001L

The method `cache` is not a pure transformation but a side-effectful operation. It just instructs the Spark interpreter to cache the dataset as soon as it's executed.  

In [56]:
val ds1: Dataset[Int] = List(1,2,3,4).toDS.map(heavyComp())
val ds1_cached: Dataset[Int] = ds1.cache

ds1: Dataset[Int] = [value: int]
ds1_cached: Dataset[Int] = [value: int]

We may expect that the only cached dataset is `ds1_cached`, but that's not true:

In [57]:
run(ds1.count)
run(ds1.count)

count at cmd56.sc:1

1 / 1

count at cmd56.sc:1

1 / 1

Took 4253


...

1 / 1

count at cmd56.sc:2

1 / 1

Took 153


res56_0: Long = 4L
res56_1: Long = 4L